In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.auto import tqdm

In [2]:
url = 'https://research.vu.nl/en/publications/'
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en,en-GB;q=0.9',
    'Cache-Control': 'no-cache',
    'Connection': 'keep-alive',
    'Cookie': 'cookie_consent_level=1; sess_auth_vu_nl=X1WOBBfsVZHB8CeQ1UrmAA|1702668317|CGX4aWAzV2EPKjeqMeN5Jq1M6ao; JSESSIONID=777824DCBBCB914C5A9EFA8D502AFD32.DS9EuTomcatA2; OptanonAlertBoxClosed=2023-12-15T18:26:17.945Z; OptanonConsent=isGpcEnabled=0&datestamp=Fri+Dec+15+2023+19%3A45%3A02+GMT%2B0100+(Central+European+Standard+Time)&version=202304.1.0&browserGpcFlag=0&isIABGlobal=false&hosts=&consentId=7911c26c-9e7b-482e-96a1-53757abbd760&interactionCount=1&landingPath=NotLandingPage&groups=1%3A1%2C2%3A0%2C4%3A0&geolocation=NL%3BNH&AwaitingReconsent=false',
    'Dnt': '1',
    'Host': 'research.vu.nl',
    'Pragma': 'no-cache',
    'Sec-Ch-Ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
    'Sec-Ch-Ua-Mobile': '?0',
    'Sec-Ch-Ua-Platform': '"Windows"',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}
payload = {
    'organisationIds': ['9626f68d-cc59-4cba-b5b7-81abca6d4d84', 'b83b6b7a-85d4-4b09-b140-cb6a5abe977f'],
    'nofollow': 'true',
    'publicationYear': ['2024', '2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010'],
    'pageSize': '100'
}

response = requests.get(url, headers=headers, params=payload)
soup = BeautifulSoup(response.text, 'html.parser')
num_pages = int(soup.findAll('a', {'class': 'step'})[-1].text)
num_pages

182

In [59]:
testing = False

In [60]:
links_FMG = []
#get num pages
payload['organisationIds'] = '9626f68d-cc59-4cba-b5b7-81abca6d4d84'
response = requests.get(url, headers=headers, params=payload)
soup = BeautifulSoup(response.text, 'html.parser')
num_pages = int(soup.findAll('a', {'class': 'step'})[-1].text)
print(num_pages)

for i in tqdm(range(1 if testing else num_pages)):
    payload['page'] = i
    response = requests.get(url, headers=headers, params=payload)
    soup = BeautifulSoup(response.text, 'html.parser')
    titles = soup.findAll('h3', {'class': 'title'})
    links = [title.find('a')['href'] for title in titles]
    links_FMG.extend(links)
    

Exception ignored in: <function tqdm.__del__ at 0x000001E37A726790>
Traceback (most recent call last):
  File "c:\Users\Sahir\anaconda3\lib\site-packages\tqdm\std.py", line 1145, in __del__
    self.close()
  File "c:\Users\Sahir\anaconda3\lib\site-packages\tqdm\notebook.py", line 286, in close
    self.disp(bar_style='success', check_delay=False)
AttributeError: 'tqdm' object has no attribute 'disp'


96


  0%|          | 0/96 [00:00<?, ?it/s]

In [61]:
links_H = []
#get num pages
payload['organisationIds'] = 'b83b6b7a-85d4-4b09-b140-cb6a5abe977f'
response = requests.get(url, headers=headers, params=payload)
soup = BeautifulSoup(response.text, 'html.parser')
num_pages = int(soup.findAll('a', {'class': 'step'})[-1].text)
print(num_pages)

for i in tqdm(range(1 if testing else num_pages)):
    payload['page'] = i
    response = requests.get(url, headers=headers, params=payload)
    soup = BeautifulSoup(response.text, 'html.parser')
    titles = soup.findAll('h3', {'class': 'title'})
    links = [title.find('a')['href'] for title in titles]
    links_H.extend(links)
    

88


  0%|          | 0/88 [00:00<?, ?it/s]

In [62]:
theses_FMG = []
for link in tqdm(links_FMG):
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.find('div', class_='rendering').text.strip()
    author = soup.find('p', class_='relations persons').text.strip()
    author = author.replace('*Corresponding author for this work', '').strip()
    author = author.split(', ')
    author = [a.replace('*', '') for a in author]
    try:
        language = soup.find('th', string='Original language').find_next_sibling('td').text.strip()
    except:
        language = None
    try:
        supervisor_row = soup.find('th', string='Supervisors/Advisors').find_next_sibling('td')
        supervisors = [s.text.strip() for s in supervisor_row.find_all('span', class_='person')]
    except:
        supervisor = None

    try:
        date = soup.find('th', string='Publication status').find_next_sibling('td').text.strip()
        #remove everything before the -
        date = date.split('-')[-1].strip()

    except:
        date = None

    try:
        faculty = "Faculty of Social Sciences"
    except:
        faculty = None


    try:
        program = soup.find('ul', class_="relations organisations")
        department = program.find('li', class_='department').text.strip()
        program = program.find('li', class_='researchprogramme').text.strip()
    except:
        program = None
        department = None

    try:
        abstract = soup.find('div', class_='rendering rendering_researchoutput rendering_researchoutput_abstractportal rendering_contributiontojournal rendering_abstractportal rendering_contributiontojournal_abstractportal').text.strip()
    except:
        abstract = None


    try:
        keywords = soup.find('ul', class_='publication-top-concepts')
        keywords_list = []
        for concept in keywords.find_all('span', class_='concept-wrapper'):
            concept_tuple = (concept.find('span', class_='concept').text.strip(), concept.find('span', class_='thesauri').text.strip(), concept.find('span', class_='value sr-only').text.strip())
            keywords_list.append(concept_tuple)
        keywords = keywords_list

    except:
        keywords = None

    try:
        journal = soup.find('th', string='Journal').find_next_sibling('td').text.strip()
    except:
        journal = None


    #make a dictionary
    thesis_dict = {
        'title': title,
        'author': author,
        'supervisors': supervisors,
        'journal': journal,
        'date': date,
        'faculty': faculty,
        'program': program,
        'department': department,
        'language': language,
        'abstract': abstract,
        'keywords': keywords
    }
    theses_FMG.append(thesis_dict)
    


  0%|          | 0/9585 [00:00<?, ?it/s]

In [63]:
#now the same for the humanities
theses_H = []
for link in tqdm(links_H):
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.find('div', class_='rendering').text.strip()
    author = soup.find('p', class_='relations persons').text.strip()
    author = author.replace('*Corresponding author for this work', '').strip()
    author = author.split(', ')
    author = [a.replace('*', '') for a in author]
    try:
        language = soup.find('th', string='Original language').find_next_sibling('td').text.strip()
    except:
        language = None
    try:
        supervisor_row = soup.find('th', string='Supervisors/Advisors').find_next_sibling('td')
        supervisors = [s.text.strip() for s in supervisor_row.find_all('span', class_='person')]
    except:
        supervisor = None

    try:
        date = soup.find('th', string='Publication status').find_next_sibling('td').text.strip()
        #remove everything before the -
        date = date.split('-')[-1].strip()

    except:
        date = None

    try:
        faculty = "Faculty of Humanities"
    except:
        faculty = None


    try:
        program = soup.find('ul', class_="relations organisations")
        department = program.find('li', class_='department').text.strip()
        program = program.find('li', class_='researchprogramme').text.strip()
    except:
        program = None
        department = None

    try:
        abstract = soup.find('div', class_='rendering rendering_researchoutput rendering_researchoutput_abstractportal rendering_contributiontojournal rendering_abstractportal rendering_contributiontojournal_abstractportal').text.strip()
    except:
        abstract = None


    try:
        keywords = soup.find('ul', class_='publication-top-concepts')
        keywords_list = []
        for concept in keywords.find_all('span', class_='concept-wrapper'):
            concept_tuple = (concept.find('span', class_='concept').text.strip(), concept.find('span', class_='thesauri').text.strip(), concept.find('span', class_='value sr-only').text.strip())
            keywords_list.append(concept_tuple)
        keywords = keywords_list

    except:
        keywords = None

    try:
        journal = soup.find('th', string='Journal').find_next_sibling('td').text.strip()
    except:
        journal = None


    #make a dictionary
    thesis_dict = {
        'title': title,
        'author': author,
        'supervisors': supervisors,
        'journal': journal,
        'date': date,
        'faculty': faculty,
        'program': program,
        'department': department,
        'language': language,
        'abstract': abstract,
        'keywords': keywords
    }
    theses_H.append(thesis_dict)
    


  0%|          | 0/8702 [00:00<?, ?it/s]

In [64]:
#make a dataframe
df_FMG = pd.DataFrame(theses_FMG)
df_H = pd.DataFrame(theses_H)

df = pd.concat([df_FMG, df_H])
df.to_csv('theses.csv', index=False)


In [5]:
import pandas as pd
df = pd.read_csv('theses.csv')
eng = df[df['language'] == 'English']
#PLOT times
doc_list = eng['abstract']
doc_list.dropna(inplace=True)
doc_list = doc_list.tolist()
len(doc_list)

C:\Users\Sahir\AppData\Local\Temp\ipykernel_15704\881048599.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  doc_list.dropna(inplace=True)


4112

In [7]:
from bertopic import BERTopic
topic_model = BERTopic(verbose=True)
topics, probs = topic_model.fit_transform(doc_list)


Batches:   0%|          | 0/129 [00:00<?, ?it/s]

2023-12-16 01:20:21,728 - BERTopic - Transformed documents to Embeddings
2023-12-16 01:21:27,459 - BERTopic - Reduced dimensionality
2023-12-16 01:21:27,961 - BERTopic - Clustered reduced embeddings


In [8]:
#visualize
topic_model.visualize_topics()

In [9]:
#hierarchy
topic_model.visualize_hierarchy()